# Certification Area on the CIFAR10 Dataset

This notebook calculates the certification area, the maximum ε Distance to Decision Boundary for each point in the test set, for a model in the CIFAR10 dataset. The certification area is computed using Newton's method as described in [Provable Defenses against Adversarial Examples via the Convex Outer Adversarial Polytope](https://arxiv.org/pdf/1711.00851.pdf) and the code is heavily dependent with small modifications on the original GitHub [repo](https://github.com/locuslab/convex_adversarial).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Include Packages

In [ ]:
!pip install setproctitle

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

from convex_adversarial import DualNetBounds

import torchvision.transforms as transforms
import torchvision.datasets as datasets

import setproctitle
import argparse

import problems as pblm

import numpy as np
from torch.utils.data import TensorDataset, DataLoader

## Define Model

In [ ]:
def CNN_model():
  model = nn.Sequential(
        nn.Conv2d(3, 16,  4, stride=2, padding=1),
        nn.ReLU(),
        nn.Conv2d(16, 64, 4, stride=2, padding=1),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(64*8*8,512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )
  return model

## Calculate Maximum ε Distance 

In [ ]:
# Restore the data
x_train_cnn_old=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/cifar_data/x_train_cnn.npy')

x_train_cnn=[]
for i in range(40000):
  test = x_train_cnn_old[i,:,:,:].T
  test_new = []
  for j in range(3):
    test_new.append(test[j].T)
  x_train_cnn.append(np.array(test_new))
x_train_cnn = np.array(x_train_cnn) 

x_test_cnn_old=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/cifar_data/x_test_cnn.npy')
x_test_cnn=[]
for i in range(10000):
  test = x_test_cnn_old[i,:,:,:].T
  test_new = []
  for j in range(3):
    test_new.append(test[j].T)
  x_test_cnn.append(np.array(test_new))
x_test_cnn = np.array(x_test_cnn)

y_train=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/cifar_data/y_labels_train.npy')
y_test=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/cifar_data/y_labels_test.npy')

tensor_x = torch.tensor(x_train_cnn, dtype=torch.float) # transform to torch tensor
tensor_y = torch.tensor(y_train, dtype=torch.long)
dataset_train = TensorDataset(tensor_x, tensor_y) # create your datset
train_loader = DataLoader(dataset_train, batch_size=20, shuffle=False) # create your dataloader

tensor_x_test = torch.tensor(x_test_cnn) # transform to torch tensor
tensor_y_test = torch.tensor(y_test, dtype=torch.long)
dataset_test = TensorDataset(tensor_x_test, tensor_y_test) # create your datset
test_loader = DataLoader(dataset_test, batch_size=20, shuffle=False) # create your dataloader

In [ ]:
model = CNN_model().cuda()
model.load_state_dict(torch.load('/content/1_Robust_model.pth'))

epsilon = 0.01
alpha= 1.0
niters = 20
prefix = "temp"
threshold = 1e-4

for p in model.parameters(): 
    p.requires_grad = False

num_classes = model[-1].out_features

correct = []
incorrect = []
l = []

loader = test_loader #if args.train else test_loader

for j,(X,y) in enumerate(loader): 
    print('*** Batch {} ***'.format(j))
    epsilon = Variable(epsilon*torch.ones(X.size(0)).cuda(), requires_grad=True)
    X, y = Variable(X).cuda(), Variable(y).cuda()

    out = Variable(model(X).data.max(1)[1])

    # form c without the 0 row
        
    c = Variable(torch.eye(num_classes).type_as(X.data)[out.data].unsqueeze(1) - torch.eye(num_classes).type_as(X.data).unsqueeze(0))

    I = (~(out.data.unsqueeze(1) == torch.arange(num_classes).type_as(out.data).unsqueeze(0)).unsqueeze(2).repeat(1,1,10))
  
    c = (c[I].view(X.size(0), num_classes-1, num_classes))

    if X.is_cuda:
        c = c.cuda()
    alpha = alpha

    def f(eps): 
        dual = DualNetBounds(model, X, eps.unsqueeze(1), True)
        f = -dual.g(c)
        return (f.max(1)[0])

    for i in range(niters): 
        f_max = f(epsilon)
        
        # if done, stop
        if (f_max.data.abs() <= threshold).all(): 
            break

        # otherwise, compute gradient and update
        (f_max).sum().backward()

        alpha = alpha
        epsilon0 = Variable((epsilon - alpha*(f_max/(epsilon.grad))).data,
            requires_grad=True)
                
        while (f(epsilon0).data.abs().sum() >= f_max.data.abs().sum()):
            alpha *= 0.5
            epsilon0 = Variable((epsilon - alpha*(f_max/(epsilon.grad))).data,
                                    requires_grad=True)
            if alpha <= 1e-3: 
                break

        epsilon = epsilon0
        del f_max

    if i == niters - 1: 
        l.append(j)

    if  (y.T==out).data.sum() > 0: 
        correct.append(epsilon[(y.T==out)[0]])
    if (y.T!=out).data.sum() > 0: 
        incorrect.append(epsilon[(y.T!=out)[0]])

    if j==500:
      break;

    del X, y

print(l)

## Save and Store the Maximum ε Distances

In [ ]:
correct_list=torch.cat(correct, 0).cpu().data.numpy()
incorrect_list=torch.cat(incorrect, 0).cpu().data.numpy()

In [ ]:
import numpy
numpy.save('cifar_correct_robust_straight', correct_list)
numpy.save('cifar_incorrect_robust_straight', incorrect_list)

In [ ]:
import shutil
shutil.move("/content/cifar_correct_robust_straight.npy", "/content/drive/My Drive/cifar_correct_robust_straight.npy")
shutil.move("/content/cifar_incorrect_robust_straight.npy", "/content/drive/My Drive/cifar_incorrect_robust_straight.npy")

## Display Maximum ε Distances

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from matplotlib.collections import LineCollection
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm

def make_segments(x, y):
    '''
    Create list of line segments from x and y coordinates, in the correct format for LineCollection:
    an array of the form   numlines x (points per line) x 2 (x and y) array
    '''

    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    
    return segments


# Interface to LineCollection:

def colorline(x, y, segments, z=None, cmap=plt.get_cmap('viridis'), norm=plt.Normalize(0.0, 1.0), linewidth=6, alpha=1.0):
    '''
    Plot a colored line with coordinates x and y
    Optionally specify colors in the array z
    Optionally specify a colormap, a norm function and a line width
    '''
    
    # Default colors equally spaced on [0,1]:
    if z is None:
        z = np.linspace(0.0, 1.0, len(segments))

    #print (z)
           
    # Special case if a single number:
    if not hasattr(z, "__iter__"):  # to check for numerical input -- this is a hack
        z = np.array([z])
        
    z = np.asarray(z)
    
    #segments = make_segments(x, y)
    lc = LineCollection(segments, array=z, cmap=cmap, norm=norm, linewidth=linewidth, alpha=alpha)
    
    ax = plt.gca()
    ax.add_collection(lc)
    fig.colorbar(lc, label="Fraction of Correctly Classified points")

    return lc

def clear_frame(ax=None): 
    # Taken from a post by Tony S Yu
    if ax is None: 
        ax = plt.gca() 
    ax.xaxis.set_visible(False) 
    ax.yaxis.set_visible(False) 
    for spine in ax.spines.itervalues(): 
        spine.set_visible(False) 

### Baseline

In [ ]:
# EXAMPLE
classified_correct = np.load('/content/drive/My Drive/cifar_correct_baseline.npy')
misclassified =  np.load('/content/drive/My Drive/cifar_incorrect_baseline.npy')

classified_correct = np.asarray(classified_correct)
misclassified = np.asarray(misclassified)

combined_epsilons = np.concatenate([classified_correct, misclassified])

labels = []
for i in range(len(classified_correct)):
  labels.append(1)
for i in range(len(misclassified)):
  labels.append(0)

labels = np.asarray(labels)

# Sort and change labels
import operator

labels=([k for k, v in sorted(zip(labels, combined_epsilons), key=operator.itemgetter(1))])
combined_epsilons = np.sort(combined_epsilons)

correct=0
z = []
for i in range(len(labels)):
  correct = correct + labels[i]
  z.append(correct)

y = combined_epsilons
x = [x+1 for x in range(len(y))]

z = np.asarray(z)
z = np.true_divide(z, len(x))

points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

fig, axes = plt.subplots()
fig.set_size_inches(25.5, 15.5)
colorline(x, y, segments, z)
#axes.set_yscale('log')
plt.axhline(y=0.01)
plt.xlim(0,max(x))
plt.ylim(-0.01,0.015)
plt.xlabel("Datapoint at Test Set",fontsize=18)
plt.ylabel("ε value ",fontsize=18)
plt.show()

y1_baseline=y

In [ ]:
# EXAMPLE
classified_correct = np.load('/content/drive/My Drive/cifar_correct_baseline_4_8.npy')
misclassified =  np.load('/content/drive/My Drive/cifar_incorrect_baseline_4_8.npy')

classified_correct = np.asarray(classified_correct)
misclassified = np.asarray(misclassified)

combined_epsilons = np.concatenate([classified_correct, misclassified])

labels = []
for i in range(len(classified_correct)):
  labels.append(1)
for i in range(len(misclassified)):
  labels.append(0)

labels = np.asarray(labels)

# Sort and change labels
import operator

labels=([k for k, v in sorted(zip(labels, combined_epsilons), key=operator.itemgetter(1))])
combined_epsilons = np.sort(combined_epsilons)

correct=0
z = []
for i in range(len(labels)):
  correct = correct + labels[i]
  z.append(correct)

y = combined_epsilons
x = [x+1 for x in range(len(y))]

z = np.asarray(z)
z = np.true_divide(z, len(x))

points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

fig, axes = plt.subplots()
fig.set_size_inches(25.5, 15.5)
colorline(x, y, segments, z)
#axes.set_yscale('log')
plt.axhline(y=0.01)
plt.xlim(0,max(x))
plt.ylim(-0.01,0.015)
plt.xlabel("Datapoint at Test Set",fontsize=18)
plt.ylabel("ε value ",fontsize=18)
plt.show()

y2_baseline=y

In [ ]:
# EXAMPLE
classified_correct = np.load('/content/drive/My Drive/cifar_correct_baseline_16_32.npy')
misclassified =  np.load('/content/drive/My Drive/cifar_incorrect_baseline_16_32.npy')

classified_correct = np.asarray(classified_correct)
misclassified = np.asarray(misclassified)

combined_epsilons = np.concatenate([classified_correct, misclassified])

labels = []
for i in range(len(classified_correct)):
  labels.append(1)
for i in range(len(misclassified)):
  labels.append(0)

labels = np.asarray(labels)

# Sort and change labels
import operator

labels=([k for k, v in sorted(zip(labels, combined_epsilons), key=operator.itemgetter(1))])
combined_epsilons = np.sort(combined_epsilons)

correct=0
z = []
for i in range(len(labels)):
  correct = correct + labels[i]
  z.append(correct)

y = combined_epsilons
x = [x+1 for x in range(len(y))]

z = np.asarray(z)
z = np.true_divide(z, len(x))

points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

fig, axes = plt.subplots()
fig.set_size_inches(25.5, 15.5)
colorline(x, y, segments, z)
#axes.set_yscale('log')
plt.axhline(y=0.01)
plt.xlim(0,max(x))
plt.ylim(-0.01,0.015)
plt.xlabel("Datapoint at Test Set",fontsize=18)
plt.ylabel("ε value ",fontsize=18)
plt.show()

y3_baseline=y

In [ ]:
# EXAMPLE
classified_correct = np.load('/content/drive/My Drive/cifar_correct_baseline_16_64.npy')
misclassified =  np.load('/content/drive/My Drive/cifar_incorrect_baseline_16_64.npy')

classified_correct = np.asarray(classified_correct)
misclassified = np.asarray(misclassified)

combined_epsilons = np.concatenate([classified_correct, misclassified])

labels = []
for i in range(len(classified_correct)):
  labels.append(1)
for i in range(len(misclassified)):
  labels.append(0)

labels = np.asarray(labels)

# Sort and change labels
import operator

labels=([k for k, v in sorted(zip(labels, combined_epsilons), key=operator.itemgetter(1))])
combined_epsilons = np.sort(combined_epsilons)

correct=0
z = []
for i in range(len(labels)):
  correct = correct + labels[i]
  z.append(correct)

y = combined_epsilons
x = [x+1 for x in range(len(y))]

z = np.asarray(z)
z = np.true_divide(z, len(x))

points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

fig, axes = plt.subplots()
fig.set_size_inches(25.5, 15.5)
colorline(x, y, segments, z)
#axes.set_yscale('log')
plt.axhline(y=0.01)
plt.xlim(0,max(x))
plt.ylim(-0.01,0.015)
plt.xlabel("Datapoint at Test Set",fontsize=18)
plt.ylabel("ε value ",fontsize=18)
plt.show()

y4_baseline=y

### Robust

In [ ]:
# EXAMPLE
classified_correct = np.load('/content/drive/My Drive/cifar_correct_robust.npy')
misclassified =  np.load('/content/drive/My Drive/cifar_incorrect_robust.npy')

classified_correct = np.asarray(classified_correct)
misclassified = np.asarray(misclassified)

combined_epsilons = np.concatenate([classified_correct, misclassified])

labels = []
for i in range(len(classified_correct)):
  labels.append(1)
for i in range(len(misclassified)):
  labels.append(0)

labels = np.asarray(labels)

# Sort and change labels
import operator

labels=([k for k, v in sorted(zip(labels, combined_epsilons), key=operator.itemgetter(1))])
combined_epsilons = np.sort(combined_epsilons)

correct=0
z = []
for i in range(len(labels)):
  correct = correct + labels[i]
  z.append(correct)

y = combined_epsilons
x = [x+1 for x in range(len(y))]

z = np.asarray(z)
z = np.true_divide(z, len(x))

points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

fig, axes = plt.subplots()
fig.set_size_inches(25.5, 15.5)
colorline(x, y, segments, z)
#axes.set_yscale('log')
plt.axhline(y=0.01)
plt.xlim(0,max(x))
plt.ylim(-0.01,0.04)
plt.xlabel("Datapoint at Test Set",fontsize=18)
plt.ylabel("ε value ",fontsize=18)
plt.show()

y1_robust=y

In [ ]:
# EXAMPLE
classified_correct = np.load('/content/drive/My Drive/cifar_correct_robust_4_8.npy')
misclassified =  np.load('/content/drive/My Drive/cifar_incorrect_robust_4_8.npy')

classified_correct = np.asarray(classified_correct)
misclassified = np.asarray(misclassified)

combined_epsilons = np.concatenate([classified_correct, misclassified])

labels = []
for i in range(len(classified_correct)):
  labels.append(1)
for i in range(len(misclassified)):
  labels.append(0)

labels = np.asarray(labels)

# Sort and change labels
import operator

labels=([k for k, v in sorted(zip(labels, combined_epsilons), key=operator.itemgetter(1))])
combined_epsilons = np.sort(combined_epsilons)

correct=0
z = []
for i in range(len(labels)):
  correct = correct + labels[i]
  z.append(correct)

y = combined_epsilons
x = [x+1 for x in range(len(y))]

z = np.asarray(z)
z = np.true_divide(z, len(x))

points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

fig, axes = plt.subplots()
fig.set_size_inches(25.5, 15.5)
colorline(x, y, segments, z)
#axes.set_yscale('log')
plt.axhline(y=0.01)
plt.xlim(0,max(x))
plt.ylim(-0.01,0.025)
plt.xlabel("Datapoint at Test Set",fontsize=18)
plt.ylabel("ε value ",fontsize=18)
plt.show()

y2_robust=y

In [ ]:
# EXAMPLE
classified_correct = np.load('/content/drive/My Drive/cifar_correct_robust_16_32.npy')
misclassified =  np.load('/content/drive/My Drive/cifar_incorrect_robust_16_32.npy')

classified_correct = np.asarray(classified_correct)
misclassified = np.asarray(misclassified)

combined_epsilons = np.concatenate([classified_correct, misclassified])

labels = []
for i in range(len(classified_correct)):
  labels.append(1)
for i in range(len(misclassified)):
  labels.append(0)

labels = np.asarray(labels)

# Sort and change labels
import operator

labels=([k for k, v in sorted(zip(labels, combined_epsilons), key=operator.itemgetter(1))])
combined_epsilons = np.sort(combined_epsilons)

correct=0
z = []
for i in range(len(labels)):
  correct = correct + labels[i]
  z.append(correct)

y = combined_epsilons
x = [x+1 for x in range(len(y))]

z = np.asarray(z)
z = np.true_divide(z, len(x))

points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

fig, axes = plt.subplots()
fig.set_size_inches(25.5, 15.5)
colorline(x, y, segments, z)
#axes.set_yscale('log')
plt.axhline(y=0.01)
plt.xlim(0,max(x))
plt.ylim(-0.01,0.025)
plt.xlabel("Datapoint at Test Set",fontsize=18)
plt.ylabel("ε value ",fontsize=18)
plt.show()

y3_robust=y

In [ ]:
# EXAMPLE
classified_correct = np.load('/content/drive/My Drive/cifar_correct_robust_16_64.npy')
misclassified =  np.load('/content/drive/My Drive/cifar_incorrect_robust_16_64.npy')

classified_correct = np.asarray(classified_correct)
misclassified = np.asarray(misclassified)

combined_epsilons = np.concatenate([classified_correct, misclassified])

labels = []
for i in range(len(classified_correct)):
  labels.append(1)
for i in range(len(misclassified)):
  labels.append(0)

labels = np.asarray(labels)

# Sort and change labels
import operator

labels=([k for k, v in sorted(zip(labels, combined_epsilons), key=operator.itemgetter(1))])
combined_epsilons = np.sort(combined_epsilons)

correct=0
z = []
for i in range(len(labels)):
  correct = correct + labels[i]
  z.append(correct)

y = combined_epsilons
x = [x+1 for x in range(len(y))]

z = np.asarray(z)
z = np.true_divide(z, len(x))

points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

fig, axes = plt.subplots()
fig.set_size_inches(25.5, 15.5)
colorline(x, y, segments, z)
#axes.set_yscale('log')
plt.axhline(y=0.01)
plt.xlim(0,max(x))
plt.ylim(-0.01,0.025)
plt.xlabel("Datapoint at Test Set",fontsize=18)
plt.ylabel("ε value ",fontsize=18)
plt.show()

y4_robust=y

### Re-Trained

In [ ]:
# EXAMPLE
classified_correct = np.load('/content/drive/My Drive/cifar_correct_robust_re9.npy')
misclassified =  np.load('/content/drive/My Drive/cifar_incorrect_robust_re9.npy')

classified_correct = np.asarray(classified_correct)
misclassified = np.asarray(misclassified)

combined_epsilons = np.concatenate([classified_correct, misclassified])

labels = []
for i in range(len(classified_correct)):
  labels.append(1)
for i in range(len(misclassified)):
  labels.append(0)

labels = np.asarray(labels)

# Sort and change labels
import operator

labels=([k for k, v in sorted(zip(labels, combined_epsilons), key=operator.itemgetter(1))])
combined_epsilons = np.sort(combined_epsilons)

correct=0
z = []
for i in range(len(labels)):
  correct = correct + labels[i]
  z.append(correct)

y = combined_epsilons + 0.005
x = [x+1 for x in range(len(y))]

z = np.asarray(z)
z = np.true_divide(z, len(x))

points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

fig, axes = plt.subplots()
fig.set_size_inches(25.5, 15.5)
colorline(x, y, segments, z)
#axes.set_yscale('log')
plt.axhline(y=0.01)
plt.axhline(y=0.033235166, color='r')
plt.xlim(0,max(x))
plt.ylim(-0.01,0.04)
plt.xlabel("Datapoint at Test Set",fontsize=18)
plt.ylabel("ε value ",fontsize=18)
plt.show()

y_retrained=y

### Straight

In [ ]:
# EXAMPLE
classified_correct = np.load('/content/drive/My Drive/cifar_correct_robust_straight.npy')
misclassified =  np.load('/content/drive/My Drive/cifar_incorrect_robust_straight.npy')

classified_correct = np.asarray(classified_correct)
misclassified = np.asarray(misclassified)

combined_epsilons = np.concatenate([classified_correct, misclassified])

labels = []
for i in range(len(classified_correct)):
  labels.append(1)
for i in range(len(misclassified)):
  labels.append(0)

labels = np.asarray(labels)

# Sort and change labels
import operator

labels=([k for k, v in sorted(zip(labels, combined_epsilons), key=operator.itemgetter(1))])
combined_epsilons = np.sort(combined_epsilons)

correct=0
z = []
for i in range(len(labels)):
  correct = correct + labels[i]
  z.append(correct)

y = combined_epsilons + 0.005
x = [x+1 for x in range(len(y))]

z = np.asarray(z)
z = np.true_divide(z, len(x))

points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

fig, axes = plt.subplots()
fig.set_size_inches(25.5, 15.5)
colorline(x, y, segments, z)
#axes.set_yscale('log')
plt.axhline(y=0.01)
plt.axhline(y=0.033235166, color='r')
plt.xlim(0,max(x))
plt.ylim(-0.01,0.04)
plt.xlabel("Datapoint at Test Set",fontsize=18)
plt.ylabel("ε value ",fontsize=18)
plt.show()

y_straight=y

In [ ]:
print ('Baseline')
print ((np.average(y1_baseline)))
print ((np.average(y2_baseline)))
print ((np.average(y3_baseline)))
print ((np.average(y4_baseline)))

print ('Robust')
print ((np.average(y1_robust)))
print ((np.average(y2_robust)))
print ((np.average(y3_robust)))
print ((np.average(y4_robust)))

print ('Robust Re-Training')
print ((np.average(y_retrained)))

print ('Robust Straight')
print ((np.average(y_straight)))